In [1]:
import pandas as pd
import numpy as np

In [128]:
pd.set_option('display.max_columns', None)
path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /2. Feature Engineering/combined_elo_df.csv'
combined_local_elo_df = pd.read_csv(path)


In [129]:
combined_local_elo_df

,Home_Team,Away_Team,Result,Link,Season,Round,League,Home_Goals,Away_Goals,Home_Win,Away_Win,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red,City,Country,Stadium,Capacity,Pitch,ELO_home,ELO_away
0,Watford,Middlesbrough,1-0,https://www.besoccer.com/match/watford-fc/midd...,2021.0,1.0,championship,1,0,True,False,2020-09-11 20:45:00,Stroud K.,1,0,0,0,Watford,England,Vicarage Road Stadium,19920,Natural,65.0,60.0
1,Watford,Luton Town,1-0,https://www.besoccer.com/match/watford-fc/luto...,2021.0,3.0,championship,1,0,True,False,2020-09-26 13:30:00,Brooks J.,0,0,2,0,Watford,England,Vicarage Road Stadium,19920,Natural,65.0,50.0
2,Watford,Blackburn Rovers,3-1,https://www.besoccer.com/match/watford-fc/blac...,2021.0,6.0,championship,3,1,True,False,2020-10-21 20:45:00,Woolmer A.,3,0,0,0,Watford,England,Vicarage Road Stadium,19920,Natural,64.0,58.0
3,Watford,AFC Bournemouth,1-1,https://www.besoccer.com/match/watford-fc/afc-...,2021.0,7.0,championship,1,1,False,False,2020-10-24 13:30:00,Robinson T.,3,0,2,0,Watford,England,Vicarage Road Stadium,19920,Natural,64.0,63.0
4,Watford,Stoke City,3-2,https://www.besoccer.com/match/watford-fc/stok...,2021.0,10.0,championship,3,2,True,False,2020-11-04 20:00:00,Davies A.,2,0,2,0,Watford,England,Vicarage Road Stadium,19920,Natural,64.0,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122546,Castel Sangro,Cesena,1-0,https://www.besoccer.com/match/castel-sangro/a...,1997.0,28.0,serie_b,1,0,True,False,1997-04-06 00:00:00,NaN,0,0,0,0,Castel di Sangro,Italy,NaN,7220,Grass,52.0,59.0
122547,Castel Sangro,Reggina,1-0,https://www.besoccer.com/match/castel-sangro/r...,1997.0,30.0,serie_b,1,0,True,False,1997-04-20 00:00:00,NaN,0,1,0,0,Castel di Sangro,Italy,NaN,7220,Grass,52.0,52.0
122548,Castel Sangro,Venezia,1-1,https://www.besoccer.com/match/castel-sangro/v...,1997.0,32.0,serie_b,1,1,False,False,1997-05-04 00:00:00,NaN,0,0,0,0,Castel di Sangro,Italy,NaN,7220,Grass,52.0,52.0
122549,Castel Sangro,Torino,2-1,https://www.besoccer.com/match/castel-sangro/t...,1997.0,34.0,serie_b,2,1,True,False,1997-05-15 00:00:00,NaN,0,1,0,1,Castel di Sangro,Italy,NaN,7220,Grass,52.0,63.0


In [140]:
def group_by_league_df(combined_local_elo_df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to group full dataframe into only the required league

    """

    league = 'premier_league'
    grouped_df = combined_local_elo_df.groupby("League")
    league_df = grouped_df.get_group(league)

    return league_df


def group_by_season_df(league_df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to group full dataframe into seasons

    """

    season = 2019.0
    grouped_df = league_df.groupby("Season")
    season_df = grouped_df.get_group(season)
    1

    return season_df

def create_summary_template(df):
    """
 
    """

    summary_df_template = pd.DataFrame(columns=
    [
        'team_name', 'season', 'league', 'total_goals_scored', 'total_goals_conceeded', 
        'largest_win_streak', 'largest_loss_streak', 'total_points', 'longest_goal_drought', 
        'total_wins'
        ])
    team_group = df.groupby("Home_Team")

    for home_team_name in df["Home_Team"].unique():
        team_df = team_group.get_group(home_team_name)
        for unique_season in team_df["Season"].unique():
            
            team_name = home_team_name
            season = unique_season
            league = team_df["League"].unique()[0]

            team_stats_dict = {
                'team_name': team_name, 'season': season, 
                'league': league, 'total_goals_scored': 0, 
                'total_goals_conceeded': 0, 'largest_win_streak': 0,
                'largest_loss_streak': 0, 'total_points': 0, 
                'longest_goal_drought': 0, 'total_wins': 0
                }
            team_stats_dict_df = pd.DataFrame([team_stats_dict])
            summary_df_template = pd.concat([summary_df_template, team_stats_dict_df], ignore_index=True)


    print("Summary Template Complete")
    return summary_df_template

def populate_season_stats(df, summary_df_template):
    for index, row in df.iterrows():

        home_team_name = row["Home_Team"]
        away_team_name = row["Away_Team"]
        season = row["Season"]
        league = row["League"]
        home_goals_scored = row["Home_Goals"]
        away_goals_scored = row["Away_Goals"]
        home_win = row["Home_Win"]
        away_win = row["Away_Win"]
            # repeated code could create a function for this
            # Home Team
        summary_df_template.loc[((summary_df_template.team_name == home_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_goals_scored"] += home_goals_scored
        
        summary_df_template.loc[((summary_df_template.team_name == home_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_goals_conceeded"] += away_goals_scored
        
        if home_goals_scored > away_goals_scored:

            summary_df_template.loc[((summary_df_template.team_name == home_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_points"] += 3
            summary_df_template.loc[((summary_df_template.team_name == home_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_wins"] += 1
        
        if home_goals_scored == away_goals_scored:

            summary_df_template.loc[((summary_df_template.team_name == home_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_points"] += 1

            # Away Team

        summary_df_template.loc[((summary_df_template.team_name == away_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_goals_scored"] += away_goals_scored
        
        summary_df_template.loc[((summary_df_template.team_name == away_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_goals_conceeded"] += home_goals_scored
        
        if away_goals_scored > home_goals_scored:

            summary_df_template.loc[((summary_df_template.team_name == away_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_points"] += 3
            summary_df_template.loc[((summary_df_template.team_name == away_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_wins"] += 1
        
        if away_goals_scored == home_goals_scored:

            summary_df_template.loc[((summary_df_template.team_name == away_team_name) & (
            summary_df_template.season == season) & (
            summary_df_template.league == league)), "total_points"] += 1

    print("Summary DF Template Populated")

    return summary_df_template

def populate_streak_season_stats(df, summary_df_populated):

    grouped_season_df = df.groupby("Season")

    for season in df["Season"].unique():

        season_df = grouped_season_df.get_group(season)
        season_df = season_df.sort_values(
            by=['Round'], ascending=[True])

        home_team_group = season_df.groupby("Home_Team")
        away_team_group = season_df.groupby("Away_Team")

        for team in season_df["Home_Team"].unique():
            home_team_df = home_team_group.get_group(team)
            away_team_df = away_team_group.get_group(team)
            team_df = pd.concat([home_team_df, away_team_df], axis=0, ignore_index=True)
            team_df = team_df.sort_values(by=['Round'], ascending=[True])
            win_streak_list = []
            win_streak = 0
            loss_streak_list = []
            loss_streak = 0
            drought_streak_list = []
            drought_streak = 0

            for index, row in team_df.iterrows():

                home_team_name = row["Home_Team"]
                away_team_name = row["Away_Team"]
                season = row["Season"]
                league = row["League"]
                home_win = row["Home_Win"]
                away_win = row["Away_Win"]
                home_goals_scored = row["Home_Goals"]
                away_goals_scored = row["Away_Goals"]

                # Winning Streak

                if (home_team_name == team and home_win == True) or (away_team_name == team and away_win == True):
                    win_streak += 1
                
                if (home_team_name == team and home_win == False) or (away_team_name == team and away_win == False):
                    win_streak = 0
                
                win_streak_list.append(win_streak)
            
                # Losing Streak     
                if (home_team_name == team and home_goals_scored < away_goals_scored) or (away_team_name == team and home_goals_scored > away_goals_scored):
                    loss_streak += 1
                if (home_team_name == team and home_goals_scored > away_goals_scored) or (away_team_name == team and home_goals_scored < away_goals_scored):
                    loss_streak = 0

                loss_streak_list.append(loss_streak)

                # Goal Drought
                if (home_team_name == team and home_goals_scored == 0) or (away_team_name == team and away_goals_scored == 0):
                    drought_streak += 1
                if (home_team_name == team and home_goals_scored > 0) or (away_team_name == team and away_goals_scored > 0):
                    drought_streak = 0
                
                drought_streak_list.append(drought_streak)

            summary_df_populated.loc[((summary_df_populated.team_name == team) & (
                summary_df_populated.season == season) & (
                summary_df_populated.league == league)), "largest_win_streak"] = max(win_streak_list)
            
            summary_df_populated.loc[((summary_df_populated.team_name == team) & (
                summary_df_populated.season == season) & (
                summary_df_populated.league == league)), "largest_loss_streak"] = max(loss_streak_list)

            summary_df_populated.loc[((summary_df_populated.team_name == team) & (
                summary_df_populated.season == season) & (
                summary_df_populated.league == league)), "longest_goal_drought"] = max(drought_streak_list)

    print("Summary DF Template Streaked")

    return summary_df_populated





In [141]:
league_df = group_by_league_df(combined_local_elo_df)
#season_df = group_by_season_df(league_df)
summary_df_template = create_summary_template(league_df)
summary_df_populated = populate_season_stats(league_df, summary_df_template)
season_df_streak_populated = populate_streak_season_stats(league_df, summary_df_populated)


In [148]:
season_df_streak_populated.sort_values(
        by=['total_wins'], ascending=[False])



,team_name,season,league,total_goals_scored,total_goals_conceeded,largest_win_streak,largest_loss_streak,total_points,longest_goal_drought,total_wins
244,Man. City,2019.0,premier_league,91,22,13,2,95,1,31
562,Liverpool,2020.0,premier_league,82,32,18,1,96,1,31
258,Man. City,2018.0,premier_league,101,25,17,1,94,1,30
450,Chelsea,2017.0,premier_league,84,33,13,2,90,1,29
553,Liverpool,2019.0,premier_league,88,22,9,1,94,1,29
...,...,...,...,...,...,...,...,...,...,...
606,Aston Villa,2016.0,premier_league,27,76,1,14,16,4,3
31,Huddersfield Town,2019.0,premier_league,22,75,1,13,16,4,3
297,Sheffield United,2021.0,premier_league,14,41,1,14,10,3,3
336,Sunderland,2006.0,premier_league,25,69,1,20,12,5,2


In [147]:
path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /2. Feature Engineering/season_df_streak_populated.csv'
season_df_streak_populated.to_csv(path, index=False)